In [4]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path: sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Segmentation and boundary-box regression

In [5]:
from src.utils.preprocess import tensor2numpy, image2axial,mask_threshold
from src.utils.utils import split_custom_filename
from src import cam
from src.classifier.agent import Agent
from src.cam import CAM_TYPES
from src import segmentation
import nibabel as nib


import numpy as np
import torchvision
import torch
import matplotlib.pyplot as plt
from src.utils.decorator import close_on_finish_decorator

## Train on ROI

In [6]:
trainer = Agent('roi2')
trainer.run()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/ROI/20210428173839
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | model          | fc               | 2.1 M 
1 | criteria       | CrossEntropyLoss | 0     
2 | metricsTracker | MetricTracker    | 0     
----------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.536     Total estimated model params size (MB)


***Defined dataloader:***
Data directory: data/SPM_categorised/Amira/
Dataset sizes - Training: 444 Validation: 112
KFold: Enabled - Fold: 1/5

Architecture [fc] was created
***Defined hyperparameters:***
"arch":                {'name': 'fc', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.1935, 0.9250, 0.9250], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'accuracy/val', 'auc/val', 'specificity/val', 'sensitivity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0}}
"roi_hparams":         {'enable': True, 'output_shape': [3, 3, 3], 'boundary_boxes': {2: [[61, 42, 73, 77, 15, 48], [10, 56, 29, 83, 5, 24], [9, 6, 38, 35, 14, 55], [4, 46, 19, 82, 35, 44], [46, 20, 72, 62, 50, 73]], 1: [[56, 18, 60, 24, 14, 19], [47, 4, 64, 17, 5, 24], [18, 20, 25, 26, 17, 22], [58, 41, 70, 60, 58, 70], [9, 56, 28,

/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
